In [1]:
import tensorflow as tf
import numpy as np
import time
from pathlib import Path
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.10.1


轉成 tflite 無量化

In [2]:
for i in range(1,7):
    model = tf.keras.models.load_model(f"./Models/Exit_Model_{i}.h5")
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    # open(f"./TFLITE_Models/Initial_Models/Exit_Model_{i}.tflite","wb").write(tflite_model)
    tflite_model_file = Path(f"./TFLITE_Models/Initial_Models/Exit_Model_{i}.tflite")
    tflite_model_file.write_bytes(tflite_model)
    
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    input_type = interpreter.get_input_details()[0]['dtype']
    print(f'model{i} input: ', input_type)
    output_type = interpreter.get_output_details()[0]['dtype']
    print(f'model{i} output: ', output_type)    

INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmp92o21ny5\assets


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmp92o21ny5\assets


model1 input:  <class 'numpy.float32'>
model1 output:  <class 'numpy.float32'>


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpfhxiu6ub\assets


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpfhxiu6ub\assets


...

INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpmxx2j9mw\assets


model5 input:  <class 'numpy.float32'>
model5 output:  <class 'numpy.float32'>


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmph7clxs9m\assets


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmph7clxs9m\assets


model6 input:  <class 'numpy.float32'>
model6 output:  <class 'numpy.float32'>


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = np.pad(train_images,[[0,0],[2,2],[2,2]],"constant",constant_values=0)
train_images = np.expand_dims(train_images,-1)
train_images = train_images.astype(np.float32) / 255.0
# print(train_images.dtype)

test_images = np.pad(test_images,[[0,0],[2,2],[2,2]],"constant",constant_values=0)
test_images = np.expand_dims(test_images,-1)
test_images = test_images.astype(np.float32) / 255.0
# print(train_images.dtype)

def evaluate(interpreter_path):
    #加载模型并分配张量
    interpreter = tf.lite.Interpreter(model_path=interpreter_path)
    interpreter.allocate_tensors()

    #获得输入输出张量.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    index = input_details[0]['index']
    shape = input_details[0]['shape']
    acc_count = 0
    image_count = test_images.shape[0]
    for i in range(image_count):
        interpreter.set_tensor(index, test_images[i].reshape(shape).astype("float32"))
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        label = np.argmax(output_data)
        if label == test_labels[i]:
            acc_count += 1
    print("test_images accuracy is {:.2%}".format(acc_count/(image_count)))

evaluate("./TFLITE_Models/Initial_Models/Exit_Model_1.tflite")

test_images accuracy is 88.40%


轉成 tflite 動態量化

In [4]:
for i in range(1,7):
    model = tf.keras.models.load_model(f"./Models/Exit_Model_{i}.h5")
    ## 动态范围量化
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model_dynamic = converter.convert()
    tflite_model_file = Path(f"./TFLITE_Models/Dynamic_Models/Exit_Model_{i}.tflite")
    tflite_model_file.write_bytes(tflite_model_dynamic)
    
    interpreter = tf.lite.Interpreter(model_content=tflite_model_dynamic)
    input_type = interpreter.get_input_details()[0]['dtype']
    print(f'model{i} input: ', input_type)
    output_type = interpreter.get_output_details()[0]['dtype']
    print(f'model{i} output: ', output_type)  


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpsm6itmnk\assets


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpsm6itmnk\assets


model1 input:  <class 'numpy.float32'>
model1 output:  <class 'numpy.float32'>


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpqoi7iodm\assets


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpqoi7iodm\assets


...

INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpdri6xrhv\assets


model5 input:  <class 'numpy.float32'>
model5 output:  <class 'numpy.float32'>


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpumjksf4k\assets


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpumjksf4k\assets


model6 input:  <class 'numpy.float32'>
model6 output:  <class 'numpy.float32'>


動態量化模型測試

In [5]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = np.pad(train_images,[[0,0],[2,2],[2,2]],"constant",constant_values=0)
train_images = np.expand_dims(train_images,-1)
train_images = train_images.astype(np.float32) / 255.0
# print(train_images.dtype)

test_images = np.pad(test_images,[[0,0],[2,2],[2,2]],"constant",constant_values=0)
test_images = np.expand_dims(test_images,-1)
test_images = test_images.astype(np.float32) / 255.0
# print(train_images.dtype)

def evaluate(interpreter_path):
    #加载模型并分配张量
    interpreter = tf.lite.Interpreter(model_path=interpreter_path)
    interpreter.allocate_tensors()

    #获得输入输出张量.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    index = input_details[0]['index']
    shape = input_details[0]['shape']
    acc_count = 0
    image_count = test_images.shape[0]
    for i in range(image_count):
        interpreter.set_tensor(index, test_images[i].reshape(shape).astype("float32"))
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        label = np.argmax(output_data)
        if label == test_labels[i]:
            acc_count += 1
    print("test_images accuracy is {:.2%}".format(acc_count/(image_count)))

evaluate("./TFLITE_Models/Dynamic_Models/Exit_Model_1.tflite")

test_images accuracy is 88.40%


轉成 tflite 部分整數型量化(輸入輸出為float32)

In [6]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images[0:100,:,:]
train_images = np.pad(train_images,[[0,0],[2,2],[2,2]],"constant",constant_values=0)
train_images = np.expand_dims(train_images,-1)
train_images = train_images.astype(np.float32) / 255.0
print(type(train_images), train_images.shape)

def representative_data_gen():
    global train_images
    for data in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
        yield [data]
        
for i in range(1,7):    
    model = tf.keras.models.load_model(f"./Models/Exit_Model_{i}.h5")
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_data_gen
    # converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    # converter.inference_input_type = tf.uint8
    # converter.inference_output_type = tf.uint8
    tflite_model_int8 = converter.convert()
    tflite_model_file = Path(f"./TFLITE_Models/Partial_Uint8_Models/Exit_Model_{i}.tflite") 
    tflite_model_file.write_bytes(tflite_model_int8)

    interpreter = tf.lite.Interpreter(model_content=tflite_model_int8)
    input_type = interpreter.get_input_details()#[0]['dtype']
    print('input: ', input_type[0]['dtype'],input_type[0]['shape'])
    output_type = interpreter.get_output_details()#[0]['dtype']
    print('output0: ', output_type[0]['dtype'],output_type[0]['shape'])
    try:
        print('output1: ', output_type[1]['dtype'],output_type[1]['shape'])
        predict, train_images = model.predict(train_images)
    except IndexError:
        predict = model.predict(train_images)
        

<class 'numpy.ndarray'> (100, 32, 32, 1)


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmp5o5wpcv7\assets


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmp5o5wpcv7\assets
C:\Users\zz408\AppData\Roaming\Python\Python39\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


input:  <class 'numpy.float32'> [ 1 32 32  1]
output0:  <class 'numpy.float32'> [ 1 10]
output1:  <class 'numpy.float32'> [ 1 28 28  6]
4/4 [==============================] - 7s 80ms/step


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmppkjk6dqw\assets


...

input:  <class 'numpy.float32'> [ 1  5  5 16]
output0:  <class 'numpy.float32'> [ 1 10]
output1:  <class 'numpy.float32'> [  1 120]
4/4 [==============================] - 0s 1ms/step


C:\Users\zz408\AppData\Roaming\Python\Python39\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpqhpkjonl\assets


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpqhpkjonl\assets
C:\Users\zz408\AppData\Roaming\Python\Python39\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


input:  <class 'numpy.float32'> [  1 120]
output0:  <class 'numpy.float32'> [ 1 10]
4/4 [==============================] - 0s 1ms/step


部分整數型量化測試

In [7]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = np.pad(train_images,[[0,0],[2,2],[2,2]],"constant",constant_values=0)
train_images = np.expand_dims(train_images,-1)
train_images = train_images.astype(np.float32) / 255.0
# print(train_images.dtype)

test_images = np.pad(test_images,[[0,0],[2,2],[2,2]],"constant",constant_values=0)
test_images = np.expand_dims(test_images,-1)
test_images = test_images.astype(np.float32) / 255.0
# print(train_images.dtype)

def evaluate(interpreter_path):
    #加载模型并分配张量
    interpreter = tf.lite.Interpreter(model_path=interpreter_path)
    interpreter.allocate_tensors()

    #获得输入输出张量.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    index = input_details[0]['index']
    shape = input_details[0]['shape']
    acc_count = 0
    image_count = test_images.shape[0]
    for i in range(image_count):
        interpreter.set_tensor(index, test_images[i].reshape(shape).astype("float32"))
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        label = np.argmax(output_data)
        if label == test_labels[i]:
            acc_count += 1
    print("test_images accuracy is {:.2%}".format(acc_count/(image_count)))

evaluate("./TFLITE_Models/Partial_Uint8_Models/Exit_Model_1.tflite")

test_images accuracy is 88.97%


轉成 tflite 全整數型量化

In [8]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images[0:100,:,:]
train_images = np.pad(train_images,[[0,0],[2,2],[2,2]],"constant",constant_values=0)
train_images = np.expand_dims(train_images,-1)
train_images = train_images.astype(np.float32) / 255.0
print(type(train_images), train_images.shape)

def representative_data_gen():
    global train_images
    for data in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
        yield [data]
        
for i in range(1,7):    
    model = tf.keras.models.load_model(f"./Models/Exit_Model_{i}.h5")
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_data_gen
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    tflite_model_int8 = converter.convert()
    tflite_model_file = Path(f"./TFLITE_Models/Full_Uint8_Models/Exit_Model_{i}.tflite") 
    tflite_model_file.write_bytes(tflite_model_int8)

    interpreter = tf.lite.Interpreter(model_content=tflite_model_int8)
    input_type = interpreter.get_input_details()#[0]['dtype']
    print('input: ', input_type[0]['dtype'],input_type[0]['shape'])
    output_type = interpreter.get_output_details()#[0]['dtype']
    print('output0: ', output_type[0]['dtype'],output_type[0]['shape'])
    try:
        print('output1: ', output_type[1]['dtype'],output_type[1]['shape'])
        predict, train_images = model.predict(train_images)
    except IndexError:
        predict = model.predict(train_images)
        

<class 'numpy.ndarray'> (100, 32, 32, 1)


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpn1fcbiup\assets


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpn1fcbiup\assets


input:  <class 'numpy.uint8'> [ 1 32 32  1]
output0:  <class 'numpy.uint8'> [ 1 10]
output1:  <class 'numpy.uint8'> [ 1 28 28  6]
4/4 [==============================] - 0s 2ms/step


C:\Users\zz408\AppData\Roaming\Python\Python39\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


...

input:  <class 'numpy.uint8'> [ 1  5  5 16]
output0:  <class 'numpy.uint8'> [ 1 10]
output1:  <class 'numpy.uint8'> [  1 120]
4/4 [==============================] - 0s 1ms/step


C:\Users\zz408\AppData\Roaming\Python\Python39\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpod9tvrvr\assets


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpod9tvrvr\assets
C:\Users\zz408\AppData\Roaming\Python\Python39\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


input:  <class 'numpy.uint8'> [  1 120]
output0:  <class 'numpy.uint8'> [ 1 10]
4/4 [==============================] - 0s 1ms/step


全整數化模型測試

In [9]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = np.pad(train_images,[[0,0],[2,2],[2,2]],"constant",constant_values=0)
train_images = np.expand_dims(train_images,-1)
# train_images = train_images.astype(np.float32) / 255.0
# print(train_images.dtype)

test_images = np.pad(test_images,[[0,0],[2,2],[2,2]],"constant",constant_values=0)
test_images = np.expand_dims(test_images,-1)
# test_images = test_images.astype(np.float32) / 255.0
# print(train_images.dtype)

def evaluate(interpreter_path):
    #加载模型并分配张量
    interpreter = tf.lite.Interpreter(model_path=interpreter_path)
    interpreter.allocate_tensors()

    #获得输入输出张量.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    index = input_details[0]['index']
    shape = input_details[0]['shape']
    acc_count = 0
    image_count = test_images.shape[0]
    for i in range(image_count):
        interpreter.set_tensor(index, test_images[i].reshape(shape).astype("uint8"))
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        label = np.argmax(output_data)
        if label == test_labels[i]:
            acc_count += 1
    print("test_images accuracy is {:.2%}".format(acc_count/(image_count)))

evaluate("./TFLITE_Models/Full_uint8_Models/Exit_Model_1.tflite")

test_images accuracy is 88.97%


轉成 tflite 半精度浮點數量化

In [10]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images[0:100,:,:]
train_images = np.pad(train_images,[[0,0],[2,2],[2,2]],"constant",constant_values=0)
train_images = np.expand_dims(train_images,-1)
train_images = train_images.astype(np.float32) / 255.0
print(type(train_images), train_images.shape)

def representative_data_gen():
    global train_images
    for data in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
        yield [data]
        
for i in range(1,7):    
    model = tf.keras.models.load_model(f"./Models/Exit_Model_{i}.h5")
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_data_gen
    converter.target_spec.supported_types = [tf.float16]

    tflite_model_float16 = converter.convert()
    tflite_model_file = Path(f"./TFLITE_Models/Float16_Models/Exit_Model_{i}.tflite") 
    tflite_model_file.write_bytes(tflite_model_float16)

    interpreter = tf.lite.Interpreter(model_content=tflite_model_float16)
    input_type = interpreter.get_input_details()#[0]['dtype']
    print('input: ', input_type[0]['dtype'],input_type[0]['shape'])
    output_type = interpreter.get_output_details()#[0]['dtype']
    print('output0: ', output_type[0]['dtype'],output_type[0]['shape'])
    try:
        print('output1: ', output_type[1]['dtype'],output_type[1]['shape'])
        predict, train_images = model.predict(train_images)
    except IndexError:
        predict = model.predict(train_images)


<class 'numpy.ndarray'> (100, 32, 32, 1)


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpahlqxv_1\assets


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpahlqxv_1\assets


input:  <class 'numpy.float32'> [ 1 32 32  1]
output0:  <class 'numpy.float32'> [ 1 10]
output1:  <class 'numpy.float32'> [ 1 28 28  6]
4/4 [==============================] - 0s 1ms/step


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmphyi7q_eo\assets


...

INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmpf9sbdxiu\assets


input:  <class 'numpy.float32'> [ 1  5  5 16]
output0:  <class 'numpy.float32'> [ 1 10]
output1:  <class 'numpy.float32'> [  1 120]
4/4 [==============================] - 0s 1ms/step


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmp01qbtas1\assets


INFO:tensorflow:Assets written to: C:\Users\zz408\AppData\Local\Temp\tmp01qbtas1\assets


input:  <class 'numpy.float32'> [  1 120]
output0:  <class 'numpy.float32'> [ 1 10]
4/4 [==============================] - 0s 2ms/step


半精度浮點數模型測試

In [11]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = np.pad(train_images,[[0,0],[2,2],[2,2]],"constant",constant_values=0)
train_images = np.expand_dims(train_images,-1)
train_images = train_images.astype(np.float32) / 255.0
# print(train_images.dtype)

test_images = np.pad(test_images,[[0,0],[2,2],[2,2]],"constant",constant_values=0)
test_images = np.expand_dims(test_images,-1)
test_images = test_images.astype(np.float32) / 255.0
# print(train_images.dtype)

def evaluate(interpreter_path):
    #加载模型并分配张量
    interpreter = tf.lite.Interpreter(model_path=interpreter_path)
    interpreter.allocate_tensors()

    #获得输入输出张量.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    index = input_details[0]['index']
    shape = input_details[0]['shape']
    acc_count = 0
    image_count = test_images.shape[0]
    for i in range(image_count):
        interpreter.set_tensor(index, test_images[i].reshape(shape).astype("float32"))
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        label = np.argmax(output_data)
        if label == test_labels[i]:
            acc_count += 1
    print("test_images accuracy is {:.2%}".format(acc_count/(image_count)))

evaluate("./TFLITE_Models/Float16_Models/Exit_Model_1.tflite")

test_images accuracy is 88.42%
